In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pathlib
from collections import defaultdict
# Defining all paths
base_dir = '/content/drive/My Drive/Colab Notebooks/food_items'
splits = ['train', 'val', 'test']
split_stats = {}

# Loop through each split
for split in splits:
    split_path = os.path.join(base_dir, split)
    class_counts = defaultdict(int)
    total_images = 0
    class_names = []

    for class_name in os.listdir(split_path):
        class_path = os.path.join(split_path, class_name)
        if os.path.isdir(class_path):
            image_files = [
                f for f in os.listdir(class_path)
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))
            ]
            count = len(image_files)
            class_counts[class_name] = count
            total_images += count
            class_names.append(class_name)

    split_stats[split] = {
        'total_images': total_images,
        'num_classes': len(class_counts),
        'class_counts': class_counts,
        'class_names': sorted(class_names)
    }

# Checking class consistency across all splits
train_classes = set(split_stats['train']['class_names'])
val_classes = set(split_stats['val']['class_names'])
test_classes = set(split_stats['test']['class_names'])

print("Split Summary:")
for split in splits:
    print(f"{split.upper()} — {split_stats[split]['num_classes']} classes, {split_stats[split]['total_images']} images")

print("\n Class Matching Check:")
print("Classes in all splits identical? ", train_classes == val_classes == test_classes)
print("Classes only in train:", train_classes - val_classes - test_classes)
print("Classes only in val:", val_classes - train_classes - test_classes)
print("Classes only in test:", test_classes - train_classes - val_classes)

Split Summary:
TRAIN — 2 classes, 600 images
VAL — 2 classes, 600 images
TEST — 2 classes, 600 images

 Class Matching Check:
Classes in all splits identical?  True
Classes only in train: set()
Classes only in val: set()
Classes only in test: set()


In [ ]:
import os
from collections import defaultdict

# Using the confirmed train directory
train_dir = '/content/drive/My Drive/Colab Notebooks/food_items/train'

# Counting the number of images per class
class_counts = defaultdict(int)

for class_name in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
        image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        class_counts[class_name] = len(image_files)

# Sorting the image count (descending)
sorted_classes = sorted(class_counts.items(), key=lambda x: x[1], reverse=True)


print("Top 20 food classes by image count:")
for name, count in sorted_classes[:20]:
    print(f"{name}: {count} images")

Top 20 food classes by image count:
main_course: 400 images
dessert: 200 images


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import shutil
from pathlib import Path
from google.colab import drive

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define source and target directories
original_base = Path("/content/drive/MyDrive/food_subset")
target_dir = Path("/content/drive/MyDrive/food_subset_shared_final")

# Step 3: Define selected categories
categories = {
    "main_course": ["chicken_curry", "hamburger", "fried_rice", "caesar_salad"],
    "dessert": ["apple_pie", "ice_cream", "carrot_cake", "baklava"]
}

# Step 4: Limit number of images per class
MAX_IMAGES = 100

# Step 5: Copy files to new structure
for split in ["train", "val", "test"]:
    src_split_path = original_base / split
    dst_split_path = target_dir / split

    for label, foods in categories.items():
        for food in foods:
            src = src_split_path / food
            dst = dst_split_path / label
            dst.mkdir(parents=True, exist_ok=True)
            if src.exists():
                files = sorted(os.listdir(src))[:MAX_IMAGES]
                for file in files:
                    full_file = src / file
                    if full_file.is_file():
                        shutil.copy(full_file, dst)

# Step 6: summary of what was saved
subset_summary = {
    split: {
        label: len(os.listdir(target_dir / split / label))
        for label in os.listdir(target_dir / split)
    }
    for split in ["train", "val", "test"]
}

subset_summary

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


{'train': {'main_course': 400, 'dessert': 200},
 'val': {'main_course': 400, 'dessert': 200},
 'test': {'dessert': 200, 'main_course': 400}}

In [ ]:
import os
import pandas as pd

base_dir = "/content/drive/MyDrive/food_subset_shared_final"
split_counts = {}

for split in ["train", "val", "test"]:
    total = 0
    split_path = os.path.join(base_dir, split)
    if not os.path.exists(split_path):
        print(f"Folder not found: {split_path}")
        continue
    for category in os.listdir(split_path):
        class_dir = os.path.join(split_path, category)
        if os.path.isdir(class_dir):
            total += len(os.listdir(class_dir))
    split_counts[split] = total

total_images = sum(split_counts.values())

if total_images == 0:
    print(" No images found in any split. Please check the folder structure.")
else:
    for split, count in split_counts.items():
        ratio = round((count / total_images) * 100, 2)
        print(f"{split.upper():<5} → {count} images ({ratio}%)")

TRAIN → 600 images (33.33%)
VAL   → 600 images (33.33%)
TEST  → 600 images (33.33%)


In [ ]:
import os
import pandas as pd

base_dir = "/content/drive/MyDrive/food_subset_shared_final"
split_counts = {}

for split in ["train", "val", "test"]:
    total = 0
    split_path = os.path.join(base_dir, split)
    if not os.path.exists(split_path):
        print(f"Folder not found: {split_path}")
        continue
    for category in os.listdir(split_path):
        class_dir = os.path.join(split_path, category)
        if os.path.isdir(class_dir):
            total += len(os.listdir(class_dir))
    split_counts[split] = total

total_images = sum(split_counts.values())

if total_images == 0:
    print(" No images found in any split. Please check the folder structure.")
else:
    for split, count in split_counts.items():
        ratio = round((count / total_images) * 100, 2)
        print(f"{split.upper():<5} → {count} images ({ratio}%)")

TRAIN → 600 images (33.33%)
VAL   → 600 images (33.33%)
TEST  → 600 images (33.33%)


In [ ]:
import os
import shutil
import random
from tqdm import tqdm

# Step 1: Set paths
original_data_dir = "/content/drive/MyDrive/food_subset_shared_final/train"
resplit_dir = "/content/drive/MyDrive/food_subset_resplit_70_10_10"


# Step 3: Clean old folder if exists
if os.path.exists(resplit_dir):
    shutil.rmtree(resplit_dir)

# Step 4: Create folder structure
splits = ["train", "val", "test"]
categories = ["main_course", "dessert"]

for split in splits:
    for category in categories:
        os.makedirs(os.path.join(resplit_dir, split, category), exist_ok=True)

# Step 5: Perform splitting
for category in tqdm(categories, desc="Processing categories"):
    category_path = os.path.join(original_data_dir, category)
    files = [f for f in os.listdir(category_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(files)

    total = len(files)
    train_end = int(0.7 * total)
    val_end = train_end + int(0.1 * total)

    split_map = {
        "train": files[:train_end],
        "val": files[train_end:val_end],
        "test": files[val_end:]
    }

    for split, split_files in split_map.items():
        for f in split_files:
            src = os.path.join(category_path, f)
            dst = os.path.join(resplit_dir, split, category, f)
            shutil.copy2(src, dst)

print(" Done: Images re-split into 70% train, 10% val, 10% test.")

Processing categories: 100%|██████████| 2/2 [00:24<00:00, 12.23s/it]

 Done: Images re-split into 70% train, 10% val, 10% test.


In [ ]:
resplit_dir = "/content/drive/MyDrive/food_subset_resplit_70_10_10"

split_counts = {}
total_images = 0

for split in ["train", "val", "test"]:
    count = 0
    split_path = os.path.join(resplit_dir, split)
    if not os.path.exists(split_path):
        print(f"Folder not found: {split_path}")
        continue
    for category in os.listdir(split_path):
        category_path = os.path.join(split_path, category)
        count += len([img for img in os.listdir(category_path) if img.lower().endswith(('.jpg', '.jpeg', '.png'))])
    split_counts[split] = count
    total_images += count

#summary
print("Final Image Split Summary:")
for split, count in split_counts.items():
    ratio = round((count / total_images) * 100, 2) if total_images else 0
    print(f"{split.upper():<5} → {count} images ({ratio}%)")

Final Image Split Summary:
TRAIN → 420 images (70.0%)
VAL   → 60 images (10.0%)
TEST  → 120 images (20.0%)


In [ ]:
# STEP 1: Install CLIP dependencies
!pip install -q ftfy regex tqdm
!pip install -q git+https://github.com/openai/CLIP.git

# STEP 2: Import libraries
import os
import clip
import torch
from PIL import Image
from tqdm import tqdm

# STEP 3: Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# STEP 4: Set paths
base_path = "/content/drive/MyDrive/food_subset_resplit_70_10_10/train"
output_path = "/content/drive/MyDrive/filtered_food_subset/train"
os.makedirs(output_path, exist_ok=True)

# STEP 5: Define valid "pure food" prompts
keep_prompts = [
    "a plate of food",
    "a cooked dish",
    "delicious meal",
    "plated meal",
    "served food"
]
text_tokens = clip.tokenize(keep_prompts).to(device)

# STEP 6: Filter images based on similarity to food prompts
for category in os.listdir(base_path):
    cat_path = os.path.join(base_path, category)
    if not os.path.isdir(cat_path):
        continue

    save_path = os.path.join(output_path, category)
    os.makedirs(save_path, exist_ok=True)

    for filename in tqdm(os.listdir(cat_path), desc=f"Filtering {category}"):
        img_path = os.path.join(cat_path, filename)
        try:
            image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
            with torch.no_grad():
                image_features = model.encode_image(image)
                text_features = model.encode_text(text_tokens)

                similarity = (image_features @ text_features.T).softmax(dim=-1)
                best_score = similarity[0].max().item()

                # Keep image only if it strongly matches "pure food"
                if best_score > 0.25:
                    Image.open(img_path).save(os.path.join(save_path, filename))
        except:
            continue

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 14.2 MB/s eta 0:00:00


100%|███████████████████████████████████████| 338M/338M [00:16<00:00, 21.4MiB/s]
Filtering dessert: 100%|██████████| 140/140 [03:05<00:00,  1.33s/it]


In [ ]:
import os
import clip
import torch
from PIL import Image
from tqdm import tqdm

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Define prompts to keep food-only images
keep_prompts = [
    "a plate of food", "a cooked dish", "delicious meal",
    "plated meal", "served food"
]
text_tokens = clip.tokenize(keep_prompts).to(device)

# Base directories
base_input_dir = "/content/drive/MyDrive/food_subset_resplit_70_10_10"
base_output_dir = "/content/drive/MyDrive/filtered_food_subset"
os.makedirs(base_output_dir, exist_ok=True)

# Only filter val and test
for split in ["val", "test"]:
    input_dir = os.path.join(base_input_dir, split)
    output_dir = os.path.join(base_output_dir, split)
    os.makedirs(output_dir, exist_ok=True)

    for category in os.listdir(input_dir):
        cat_path = os.path.join(input_dir, category)
        if not os.path.isdir(cat_path):
            continue

        save_path = os.path.join(output_dir, category)
        os.makedirs(save_path, exist_ok=True)

        for filename in tqdm(os.listdir(cat_path), desc=f"Filtering {split}/{category}"):
            img_path = os.path.join(cat_path, filename)
            try:
                image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
                with torch.no_grad():
                    image_features = model.encode_image(image)
                    text_features = model.encode_text(text_tokens)
                    similarity = (image_features @ text_features.T).softmax(dim=-1)
                    best_score = similarity[0].max().item()

                    if best_score > 0.25:
                        Image.open(img_path).save(os.path.join(save_path, filename))
            except:
                continue

Filtering test/main_course: 100%|██████████| 80/80 [02:37<00:00,  1.96s/it]


In [ ]:
filtered_base = "/content/drive/MyDrive/filtered_food_subset"

def check_split_ratios(base_dir):
    split_counts = {}
    total_images = 0

    for split in ["train", "val", "test"]:
        split_path = os.path.join(base_dir, split)
        image_count = 0

        if not os.path.exists(split_path):
            print(f" Missing folder: {split_path}")
            continue

        for category in os.listdir(split_path):
            category_path = os.path.join(split_path, category)
            if os.path.isdir(category_path):
                count = len([
                    f for f in os.listdir(category_path)
                    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
                ])
                image_count += count

        split_counts[split] = image_count
        total_images += image_count

    print(f"\n Total images across all splits: {total_images}\n")
    for split, count in split_counts.items():
        percent = (count / total_images * 100) if total_images > 0 else 0
        print(f"{split.upper():<5} → {count} images ({percent:.2f}%)")

# Run the check
check_split_ratios(filtered_base)


 Total images across all splits: 1393

TRAIN → 600 images (43.07%)
VAL   → 321 images (23.04%)
TEST  → 472 images (33.88%)


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
import numpy as np
import os

# Define paths
train_dir = '/content/drive/MyDrive/filtered_food_subset/train'
val_dir = '/content/drive/MyDrive/filtered_food_subset/val'


# Check if directories exist and contain files
if not os.path.exists(train_dir) or not any(os.scandir(train_dir)):
    raise FileNotFoundError(f"Training directory not found or is empty: {train_dir}. Please ensure the filtering process successfully saved images to this location.")

if not os.path.exists(val_dir) or not any(os.scandir(val_dir)):
     raise FileNotFoundError(f"Validation directory not found or is empty: {val_dir}. Please ensure the filtering process successfully saved images to this location.")


# Improved data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Calculate class weights
# Use train_gen.classes which is populated after flow_from_directory
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)
class_weights_dict = dict(enumerate(class_weights))


# Load MobileNetV2 base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tune last 30 layers instead of 10
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(64, activation='relu')(x)  # Added layer for deeper learning
output = Dense(2, activation='softmax')(x)  # Adjust if more than 2 classes

# Final model
mobilenet_model = Model(inputs=base_model.input, outputs=output)

# Compile model
mobilenet_model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
]

# Train model with class weights
mobilenet_model.fit(train_gen, validation_data=val_gen, epochs=25, callbacks=callbacks, class_weight=class_weights_dict)

# Save model
mobilenet_model.save('/content/drive/MyDrive/filtered_food_subset/mobilenetv2_model.h5')

Found 600 images belonging to 2 classes.
Found 321 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5151 - loss: 0.9211

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 180s 8s/step - accuracy: 0.5180 - loss: 0.9168 - val_accuracy: 0.8660 - val_loss: 0.3906 - learning_rate: 1.0000e-04
Epoch 2/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 79s 4s/step - accuracy: 0.8103 - loss: 0.4006 - val_accuracy: 0.8972 - val_loss: 0.2529 - learning_rate: 1.0000e-04
Epoch 3/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - accuracy: 0.9010 - loss: 0.2220 - val_accuracy: 0.9128 - val_loss: 0.2034 - learning_rate: 1.0000e-04
Epoch 4/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 70s 4s/step - accuracy: 0.8920 - loss: 0.2309 - val_accuracy: 0.9221 - val_loss: 0.1683 - learning_rate: 1.0000e-04
Epoch 5/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 66s 4s/step - accuracy: 0.9261 - loss: 0.1686 - val_accuracy: 0.9315 - val_loss: 0.1453 - learning_rate: 1.0000e-04
Epoch 6/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 66s 4s/step - accuracy: 0.9376 - loss: 0.1355 - val_accuracy: 0.9408 - val_loss: 0.1248 - learning_rate: 1.0000e-04
Epoch 7/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 76s 4s/step - accuracy: 0.9553 - loss: 0.1559 - 

In [ ]:
print("Class distribution in the training set:")
display(split_stats['train']['class_counts'])

Class distribution in the training set:


defaultdict(int, {'main_course': 400, 'dessert': 200})

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf

# Load the trained MobileNetV2 model
model = tf.keras.models.load_model('/content/drive/MyDrive/filtered_food_subset/mobilenetv2_model.h5')

# Define class_names using the class_indices from the training generator
# This ensures the order matches the model's output
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) # Create a simple generator to get class_indices
train_gen = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/filtered_food_subset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
class_names = list(train_gen.class_indices.keys())

print(f"Class names in order: {class_names}")


def predict_image_debug(image_path):
  # Load the image from the provided path
  img = Image.open(image_path)
  img_array = np.array(img)
  print(f"Input image shape: {img_array.shape}")

  # Resize the image to the target size (224, 224) - Model was trained on 224x224
  img_resized = tf.image.resize(img_array, (224, 224))
  print(f"Resized image shape: {img_resized.shape}")

  img_4d = np.expand_dims(img_resized, axis=0)  # Add batch dimension
  print(f"Image shape with batch dimension: {img_4d.shape}")

  prediction = model.predict(img_4d)[0]
  print(f"Raw prediction output: {prediction}")

  return {class_names[i]: float(prediction[i]) for i in range(len(class_names))}

# Replace 'path/to/your/dessert_image.jpg' with the actual path to your image
predict_image_debug('/content/drive/MyDrive/filtered_food_subset/test/dessert/104465.jpg')

Found 600 images belonging to 2 classes.
Class names in order: ['dessert', 'main_course']
Input image shape: (512, 384, 3)
Resized image shape: (224, 224, 3)
Image shape with batch dimension: (1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Raw prediction output: [0.23219042 0.76780957]


{'dessert': 0.23219041526317596, 'main_course': 0.7678095698356628}

In [ ]:
import gradio as gr
import numpy as np
import tensorflow as tf
from PIL import Image

# Assuming the model and class_names are already loaded and defined
# model = tf.keras.models.load_model('/content/drive/MyDrive/filtered_food_subset/mobilenetv2_model.h5')
# class_names = ['dessert', 'main_course'] # Ensure this matches the model's output order

def classify_image(image):
  # The input 'image' is a PIL Image object from Gradio
  img_array = np.array(image)
  # Ensure image is in RGB format if it's grayscale or has an alpha channel
  if img_array.shape[-1] == 4:
      img_array = img_array[:, :, :3]
  elif len(img_array.shape) == 2:
      img_array = np.stack((img_array,) * 3, axis=-1)

  img_resized = tf.image.resize(img_array, (224, 224))
  img_4d = np.expand_dims(img_resized, axis=0)  # Add batch dimension
  img_4d = img_4d / 255.0 # Rescale the image

  prediction = model.predict(img_4d)[0]

  return {class_names[i]: float(prediction[i]) for i in range(len(class_names))}

# Define class_names using the class_indices from the training generator to ensure correct order
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) # Create a simple generator to get class_indices
train_gen = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/filtered_food_subset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
class_names = list(train_gen.class_indices.keys())


image_input = gr.Image(label="Upload Food Image")
label_output = gr.Label(num_top_classes=2)

gr.Interface(fn=classify_image, inputs=image_input, outputs=label_output).launch(debug=True)

Found 600 images belonging to 2 classes.
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4c6abf5a10de3edb6f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


In [ ]:
custom_css = """
body {
  background-color: #f0f0f0; /* Change this to your desired background color */
}
"""

Now, update your `gr.Interface` call to include the `css` argument:

In [ ]:
# Brainstorming typical restaurant menu elements and mapping to Gradio components:
# Menu Sections: Appetizers, Main Courses, Desserts, Drinks -> Can be represented by gr.Tabs or separate sections within gr.Blocks using rows/columns.
# Dish Name: -> gr.Label or gr.Textbox (display only)
# Dish Description: -> gr.Textbox (display only)
# Dish Image: -> gr.Image
# Dish Price: -> gr.Label or gr.Number (display only)

# Considering using gr.Blocks for flexible layout:
# Use gr.Blocks to create the overall page structure.
# Use gr.Tabs for different menu sections (Appetizers, Main Courses, etc.).
# Within each tab (or section), use gr.Column or gr.Row to arrange individual menu items.
# For each menu item, use a combination of gr.Image, gr.Label, and gr.Textbox to display the image, name, description, and price.

# Outline of the Gradio interface structure:
# gr.Blocks:
#   gr.HTML (Optional: for a title/header)
#   gr.Tabs:
#     gr.Tab (label="Appetizers"):
#       gr.Row: # For the first appetizer
#         gr.Image
#         gr.Column:
#           gr.Label (Dish Name)
#           gr.Textbox (Dish Description)
#           gr.Label (Dish Price)
#       gr.Row: # For the second appetizer
#         gr.Image
#         gr.Column:
#           gr.Label (Dish Name)
#           gr.Textbox (Dish Description)
#           gr.Label (Dish Price)
#       # Add more Rows for other appetizers
#     gr.Tab (label="Main Courses"):
#       # Similar structure as Appetizers tab
#     gr.Tab (label="Desserts"):
#       # Similar structure as Appetizers tab
#     gr.Tab (label="Drinks"):
#       # Similar structure as Appetizers tab
#   gr.HTML (Optional: for a footer)

print("Gradio interface structure outlined.")

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf

# Load the trained model (assuming this is already loaded in your environment, but including for clarity)
# model = tf.keras.models.load_model('/content/drive/MyDrive/filtered_food_subset/mobilenetv2_model.h5')

# Define class_names using the class_indices from the training generator
# This ensures the order matches the model's output
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) # Create a simple generator to get class_indices
train_gen = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/filtered_food_subset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
class_names = list(train_gen.class_indices.keys())

print(f"Class names in order: {class_names}")


def predict_image_debug(image_path):
  # Load the image from the provided path
  img = Image.open(image_path)
  img_array = np.array(img)
  print(f"Input image shape: {img_array.shape}")

  # Resize the image to the target size (224, 224) - Model was trained on 224x224
  img_resized = tf.image.resize(img_array, (224, 224))
  print(f"Resized image shape: {img_resized.shape}")

  img_4d = np.expand_dims(img_resized, axis=0)  # Add batch dimension
  print(f"Image shape with batch dimension: {img_4d.shape}")

  prediction = model.predict(img_4d)[0]
  print(f"Raw prediction output: {prediction}")

  return {class_names[i]: float(prediction[i]) for i in range(len(class_names))}

# Replace 'path/to/your/dessert_image.jpg' with the actual path to your image
predict_image_debug('/content/drive/MyDrive/filtered_food_subset/test/dessert/104465.jpg')

In [ ]:
print("Class indices from the training generator:")
display(train_gen.class_indices)

In [ ]:
import gradio as gr
import numpy as np
import tensorflow as tf
from PIL import Image

# Assuming the model and class_names are already loaded and defined
# model = tf.keras.models.load_model('/content/drive/MyDrive/filtered_food_subset/mobilenetv2_model.h5')
# class_names = ['dessert', 'main_course'] # Ensure this matches the model's output order

def classify_image(image):
  # The input 'image' is a PIL Image object from Gradio
  img_array = np.array(image)
  # Ensure image is in RGB format if it's grayscale or has an alpha channel
  if img_array.shape[-1] == 4:
      img_array = img_array[:, :, :3]
  elif len(img_array.shape) == 2:
      img_array = np.stack((img_array,) * 3, axis=-1)

  img_resized = tf.image.resize(img_array, (224, 224))
  img_4d = np.expand_dims(img_resized, axis=0)  # Add batch dimension
  img_4d = img_4d / 255.0 # Rescale the image

  prediction = model.predict(img_4d)[0]

  return {class_names[i]: float(prediction[i]) for i in range(len(class_names))}

# Define class_names using the class_indices from the training generator to ensure correct order
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) # Create a simple generator to get class_indices
train_gen = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/filtered_food_subset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
class_names = list(train_gen.class_indices.keys())


image_input = gr.Image(label="Upload Food Image")
label_output = gr.Label(num_top_classes=2)

gr.Interface(fn=classify_image, inputs=image_input, outputs=label_output).launch(debug=True)

Found 600 images belonging to 2 classes.
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b03f74abe1efb75000.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1702, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b03f74abe1efb75000.gradio.live
